## Final Notebook Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from statsmodels.formula.api import ols
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
housing_df = pd.read_csv('data/kc_house_data.csv')
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  object 
 9   view           21534 non-null  object 
 10  condition      21597 non-null  object 
 11  grade          21597 non-null  object 
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [3]:
housing_df.isna().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64

In [3]:
housing_df['datetime'] = pd.to_datetime(housing_df['date'])
housing_df['soldyear'] = housing_df['datetime'].dt.year
housing_df['age_when_sold'] = housing_df['soldyear'] - housing_df['yr_built']

In [4]:
housing_df['grade_num'] = housing_df['grade'].str.split()
housing_df['grade_num'] = housing_df['grade_num'].str[0]
housing_df['grade_num'] = housing_df['grade_num'].astype(int)

In [5]:
housing_df['sqft_basement'] = housing_df['sqft_living'] - housing_df['sqft_above']
housing_df['Basement'] = None
housing_df['Basement'] = housing_df['sqft_basement'].map(lambda x: False if x == 0 else True)
housing_df['Basement'] = housing_df['Basement'].astype(int)

In [7]:
# replace_dict = {0.75: 1, 1.75: 2, 2.75: 3, 3.75: 4, 4.75: 5, 5.75: 6, 6.75: 7, 7.75: 8, 
#                 1.25: 1.5, 2.25: 2.5, 3.25: 3.5, 4.25: 4.5, 5.25: 5.5, 6.25: 6.5}
# housing_df['bathrooms'] = housing_df['bathrooms'].replace(replace_dict)

In [8]:
housing_df.condition.value_counts()

Average      14020
Good          5677
Very Good     1701
Fair           170
Poor            29
Name: condition, dtype: int64

In [6]:
replace_dict2 = {'Poor': 1, 'Fair': 2, 'Average': 3, 'Good': 4, 'Very Good': 5}
housing_df['condition'] = housing_df['condition'].replace(replace_dict2)

In [7]:
from sklearn.impute import SimpleImputer
wtr_col = housing_df[['waterfront']]
imputer = SimpleImputer(strategy='constant', fill_value = 'NO')
imputer.fit(wtr_col)
waterfront_imputed = imputer.transform(wtr_col)
housing_df.waterfront = waterfront_imputed

In [8]:
from sklearn.preprocessing import OrdinalEncoder
wtr_col = housing_df[['waterfront']]
encoder_wtr = OrdinalEncoder()
encoder_wtr.fit(wtr_col)
encoded_wtr = encoder_wtr.transform(wtr_col)
encoded_wtr = encoded_wtr.flatten()
housing_df.waterfront = encoded_wtr

In [9]:
from sklearn.impute import SimpleImputer
view_col = housing_df[['view']]
imputer = SimpleImputer(strategy='constant', fill_value = 'NONE')
imputer.fit(view_col)
view_imputed = imputer.transform(view_col)
housing_df.view = view_imputed

In [13]:
housing_df.isna().sum()

id                  0
date                0
price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront          0
view                0
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
datetime            0
soldyear            0
age_when_sold       0
grade_num           0
Basement            0
dtype: int64

In [10]:
housing_ols = housing_df.drop(['datetime', 'date', 'soldyear', 'yr_built', 'lat', 'long', 'sqft_above', 'sqft_lot15', 'sqft_living15', 'grade', 'sqft_above', 'sqft_basement', 'yr_renovated'], axis = 1)

In [15]:
q_4_house = housing_ols["bedrooms"].value_counts().quantile(1)
q_3_house = housing_ols["bedrooms"].value_counts().quantile(.75)
q_2_house = housing_ols["bedrooms"].value_counts().quantile(.5)
q_1_house = housing_ols["bedrooms"].value_counts().quantile(.25)
q_1_house, q_2_house, q_3_house, q_4_house

(5.25, 117.0, 1890.75, 9824.0)

In [16]:
housing_ols['iqr_95_bed'] = housing_ols['bedrooms'][housing_ols['bedrooms'].between(housing_ols['bedrooms'].quantile(.025), housing_ols['bedrooms'].quantile(.975), inclusive=True)]

In [17]:
housing_ols.head()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,zipcode,age_when_sold,grade_num,Basement,iqr_95_bed
0,7129300520,221900.0,3,1.00,1180,5650,1.0,0.0,NONE,3,98178,59,7,0,3.0
1,6414100192,538000.0,3,2.25,2570,7242,2.0,0.0,NONE,3,98125,63,7,1,3.0
2,5631500400,180000.0,2,1.00,770,10000,1.0,0.0,NONE,3,98028,82,6,0,2.0
3,2487200875,604000.0,4,3.00,1960,5000,1.0,0.0,NONE,5,98136,49,7,1,4.0
4,1954400510,510000.0,3,2.00,1680,8080,1.0,0.0,NONE,3,98074,28,8,0,3.0


In [18]:
# housing_ols.dropna(subset =['iqr_95_be'],  inplace=True)

In [19]:
len(housing_ols['iqr_95_bed'])

21597

In [20]:
housing_ols.drop(['id', 'bedrooms', 'bathrooms', 'floors', 'zipcode', 'grade_num', 'waterfront', 'age_when_sold', 'Basement', 'condition', 'sqft_lot', 'iqr_95_bed'], axis =1, inplace =True)

In [ ]:
len(housing_ols)

In [ ]:
housing_ols['bathrooms'] = housing_ols['bathrooms'][housing_ols['bathrooms'].between(housing_ols['bathrooms'].quantile(.025), housing_ols['bathrooms'].quantile(.975), inclusive=True)]
housing_ols['floors'] = housing_ols['floors'][housing_ols['floors'].between(housing_ols['floors'].quantile(.025), housing_ols['floors'].quantile(.975), inclusive=True)]
housing_ols['waterfront'] = housing_ols['waterfront'][housing_ols['waterfront'].between(housing_ols['waterfront'].quantile(.025), housing_ols['waterfront'].quantile(.975), inclusive=True)]
housing_ols['condition'] = housing_ols['condition'][housing_ols['condition'].between(housing_ols['condition'].quantile(.025), housing_ols['condition'].quantile(.975), inclusive=True)]
housing_ols['age_when_sold'] = housing_ols['age_when_sold'][housing_ols['age_when_sold'].between(housing_ols['age_when_sold'].quantile(.025), housing_ols['age_when_sold'].quantile(.975), inclusive=True)]
housing_ols['grade_num'] = housing_ols['grade_num'][housing_ols['grade_num'].between(housing_ols['grade_num'].quantile(.025), housing_ols['grade_num'].quantile(.975), inclusive=True)]
housing_ols['Basement'] = housing_ols['Basement'][housing_ols['Basement'].between(housing_ols['Basement'].quantile(.025), housing_ols['Basement'].quantile(.975), inclusive=True)]



In [ ]:
housing_ols['iqr_95_Basement'] = housing_ols['Basement'][housing_ols['Basement'].between(housing_ols['Basement'].quantile(.025), housing_ols['Basement'].quantile(.975), inclusive=True)]


In [ ]:
# housing_ols['iqr_95_Basement']

In [113]:
housing_ols['iqr_95_sqft_living'] = housing_ols['sqft_living'][housing_ols['sqft_living'].between(housing_ols['sqft_living'].quantile(.025), housing_ols['sqft_living'].quantile(.975), inclusive=True)]
housing_ols['iqr_90_sqft_living'] = housing_ols['sqft_living'][housing_ols['sqft_living'].between(housing_ols['sqft_living'].quantile(.05), housing_ols['sqft_living'].quantile(.95), inclusive=True)]
housing_ols['iqr_97.5_sqft_living'] = housing_ols['sqft_living'][housing_ols['sqft_living'].between(housing_ols['sqft_living'].quantile(.0125), housing_ols['sqft_living'].quantile(.9875), inclusive=True)]
housing_ols['iqr_99_sqft_living'] = housing_ols['sqft_living'][housing_ols['sqft_living'].between(housing_ols['sqft_living'].quantile(.01), housing_ols['sqft_living'].quantile(.99), inclusive=True)]
housing_ols['iqr_92.5_sqft_living'] = housing_ols['sqft_living'][housing_ols['sqft_living'].between(housing_ols['sqft_living'].quantile(.0375), housing_ols['sqft_living'].quantile(.9625), inclusive=True)]
housing_ols['iqr_99.9_sqft_living'] = housing_ols['sqft_living'][housing_ols['sqft_living'].between(housing_ols['sqft_living'].quantile(.001), housing_ols['sqft_living'].quantile(.999), inclusive=True)]
housing_ols['iqr_99.99_sqft_living'] = housing_ols['sqft_living'][housing_ols['sqft_living'].between(housing_ols['sqft_living'].quantile(.0001), housing_ols['sqft_living'].quantile(.9999), inclusive=True)]


In [ ]:
# housing_ols.plot(kind ='scatter', x = 'iqr_90_sqft_living', y = 'price',alpha =.4, color = 'b')
# housing_ols.plot(kind ='scatter', x = 'iqr_92.5_sqft_living', y = 'price',alpha =.4, color = 'b')
# housing_ols.plot(kind ='scatter', x = 'iqr_95_sqft_living', y = 'price',alpha =.4, color = 'b')
# housing_ols.plot(kind ='scatter', x = 'iqr_97.5_sqft_living', y = 'price',alpha =.4, color = 'b')
# housing_ols.plot(kind ='scatter', x = 'iqr_99_sqft_living', y = 'price',alpha =.4, color = 'b')
# housing_ols.plot(kind ='scatter', x = 'sqft_living', y = 'price',alpha =.4, color = 'b')

In [ ]:
# housing_ols['iqr_99.99_sqft_living'] = housing_ols['sqft_living'][housing_ols['sqft_living'].between(housing_ols['sqft_living'].quantile(.0001), housing_ols['sqft_living'].quantile(.9999), inclusive=True)]
# housing_ols['iqr_99.9_sqft_living'] = housing_ols['sqft_living'][housing_ols['sqft_living'].between(housing_ols['sqft_living'].quantile(.001), housing_ols['sqft_living'].quantile(.999), inclusive=True)]

# # housing_ols.dropna(subset= ['iqr_99.9_sqft_living'], inplace = True)
# housing_ols

In [ ]:
# housing_ols['iqr_99.9_sqft_living'] = housing_ols['sqft_living'][housing_ols['sqft_living'].between(housing_ols['sqft_living'].quantile(.001), housing_ols['sqft_living'].quantile(.999), inclusive=True)]
# housing_ols.dropna(subset= ['iqr_99.9_sqft_living'], inplace = True)
# housing_ols

In [114]:
housing_ols['iqr_99.9_price'] = housing_ols['price'][housing_ols['price'].between(housing_ols['price'].quantile(.001), housing_ols['price'].quantile(.999), inclusive=True)]


In [ ]:
housing_ols['iqr_99.9_price'].corr(housing_ols['iqr_99.9_sqft_living'])

In [117]:
housing_ols.drop(['price'], axis=1, inplace =True)

In [118]:
housing_ols.corr()

,sqft_living,iqr_95_sqft_living,iqr_90_sqft_living,iqr_97.5_sqft_living,iqr_99_sqft_living,iqr_92.5_sqft_living,iqr_99.9_sqft_living,iqr_99.99_sqft_living,iqr_99.9_price
sqft_living,1.000000,1.000000,1.00000,1.000000,1.000000,1.00000,1.000000,1.000000,0.693463
iqr_95_sqft_living,1.000000,1.000000,1.00000,1.000000,1.000000,1.00000,1.000000,1.000000,0.624188
iqr_90_sqft_living,1.000000,1.000000,1.00000,1.000000,1.000000,1.00000,1.000000,1.000000,0.579960
iqr_97.5_sqft_living,1.000000,1.000000,1.00000,1.000000,1.000000,1.00000,1.000000,1.000000,0.650871
iqr_99_sqft_living,1.000000,1.000000,1.00000,1.000000,1.000000,1.00000,1.000000,1.000000,0.656746
iqr_92.5_sqft_living,1.000000,1.000000,1.00000,1.000000,1.000000,1.00000,1.000000,1.000000,0.601460
iqr_99.9_sqft_living,1.000000,1.000000,1.00000,1.000000,1.000000,1.00000,1.000000,1.000000,0.689368
iqr_99.99_sqft_living,1.000000,1.000000,1.00000,1.000000,1.000000,1.00000,1.000000,1.000000,0.693462
iqr_99.9_price,0.693463,0.624188,0.57996,0.650871,0.656746,0.60146,0.689368,0.693462,1.000000


In [ ]:
# mse can be ginormous which means wrong algorithm but can still show and can say don't use this model

In [66]:
housing_ols.dropna(subset=['iqr_99.99_sqft_living'],inplace=True)

In [68]:
housing_ols.corr()

,price,sqft_living,iqr_95_bed,iqr_95_sqft_living,iqr_90_sqft_living,iqr_97.5_sqft_living,iqr_99_sqft_living,iqr_92.5_sqft_living,iqr_99.9_sqft_living,iqr_99.99_sqft_living
price,1.000000,0.699543,0.309198,0.624551,0.580113,0.651385,0.657085,0.601736,0.691027,0.699543
sqft_living,0.699543,1.000000,0.583856,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
iqr_95_bed,0.309198,0.583856,1.000000,0.568451,0.540185,0.582244,0.585657,0.553135,0.585160,0.583856
iqr_95_sqft_living,0.624551,1.000000,0.568451,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
iqr_90_sqft_living,0.580113,1.000000,0.540185,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
iqr_97.5_sqft_living,0.651385,1.000000,0.582244,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
iqr_99_sqft_living,0.657085,1.000000,0.585657,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
iqr_92.5_sqft_living,0.601736,1.000000,0.553135,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
iqr_99.9_sqft_living,0.691027,1.000000,0.585160,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
iqr_99.99_sqft_living,0.699543,1.000000,0.583856,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [69]:
housing_ols.dropna(subset=['iqr_90_sqft_living'],inplace=True)
housing_ols.corr()

,price,sqft_living,iqr_95_bed,iqr_95_sqft_living,iqr_90_sqft_living,iqr_97.5_sqft_living,iqr_99_sqft_living,iqr_92.5_sqft_living,iqr_99.9_sqft_living,iqr_99.99_sqft_living
price,1.000000,0.580113,0.226935,0.580113,0.580113,0.580113,0.580113,0.580113,0.580113,0.580113
sqft_living,0.580113,1.000000,0.540185,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
iqr_95_bed,0.226935,0.540185,1.000000,0.540185,0.540185,0.540185,0.540185,0.540185,0.540185,0.540185
iqr_95_sqft_living,0.580113,1.000000,0.540185,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
iqr_90_sqft_living,0.580113,1.000000,0.540185,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
iqr_97.5_sqft_living,0.580113,1.000000,0.540185,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
iqr_99_sqft_living,0.580113,1.000000,0.540185,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
iqr_92.5_sqft_living,0.580113,1.000000,0.540185,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
iqr_99.9_sqft_living,0.580113,1.000000,0.540185,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
iqr_99.99_sqft_living,0.580113,1.000000,0.540185,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=7, figsize=(16,3))
for xcol, ax in zip(['sqft_living', 'sqft_lot', 'bedrooms', 'bathrooms', 'floors', 'waterfront', 'grade_num'], axes):
    housing_ols.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha=0.4, color='b')

In [ ]:
correlation = housing_ols['price']. corr(housing_ols['grade_num']) 
correlation

In [ ]:
housing_ols.corr()

In [ ]:
xx = housing_ols.corr()
x = xx[['price']]
sns.heatmap(x, annot=True, linewidths =2.5)

In [ ]:
# formula_price_grade = 'price ~C(grade_num) +sqft_living + C(bedrooms) + C(bathrooms) + sqft_lot+ C(floors) + C(waterfront) + C(condition) + C(zipcode) + age_when_sold + C(Basement)'
# formula_price_grade_model = ols(formula = formula_price_grade, data = housing_ols).fit()
# formula_price_grade_model_summary = formula_price_grade_model.summary()
# formula_price_grade_model_summary

In [11]:
housing_pred = housing_ols.copy()
bedrooms_ohe = housing_pred[['bedrooms']]
ohe_bedrooms = OneHotEncoder(categories ='auto', sparse =False)
ohe_bedrooms.fit(bedrooms_ohe)
ohe_bedrooms_encoded = ohe_bedrooms.transform(bedrooms_ohe)
bedrooms_encoded_ohe = pd.DataFrame(ohe_bedrooms_encoded, columns = ohe_bedrooms.get_feature_names(['bedrooms']), index = housing_pred.index )
housing_pred1 = pd.concat([housing_pred, bedrooms_encoded_ohe ], axis =1)

In [12]:
grade_num_ohe = housing_pred[['grade_num']]
ohe_grade_num = OneHotEncoder(categories ='auto', sparse =False)
ohe_grade_num.fit(grade_num_ohe)
ohe_grade_num_encoded = ohe_grade_num.transform(grade_num_ohe)
grade_num_encoded_ohe = pd.DataFrame(ohe_grade_num_encoded, columns = ohe_grade_num.get_feature_names(['grade_num']), index = housing_pred.index )
housing_pred2 = pd.concat([housing_pred1, grade_num_encoded_ohe], axis =1)

In [13]:
condition_ohe = housing_pred[['condition']]
ohe = OneHotEncoder(categories="auto", sparse=False)
cond_encoded_ohe = pd.DataFrame (ohe.fit_transform(condition_ohe))
cond_encoded_ohe.columns = ohe.get_feature_names(['condition'])
housing_pred3 = pd.concat([housing_pred2, cond_encoded_ohe], axis = 1)

In [14]:
bathrooms_ohe = housing_pred[['bathrooms']]
ohe = OneHotEncoder(categories='auto', sparse=False, handle_unknown='ignore')
bathrooms_transform = ohe.fit_transform(bathrooms_ohe)
bathrooms_encoded_ohe = pd.DataFrame(bathrooms_transform, columns=ohe.get_feature_names(['bathrooms']), index=housing_pred.index)
housing_pred4 = pd.concat([housing_pred3, bathrooms_encoded_ohe], axis = 1)

In [15]:
view_ohe = housing_pred[['view']]
ohe = OneHotEncoder(categories="auto", sparse=False)
ohe.fit(view_ohe)
view_encoded = ohe.transform(view_ohe)
view_encoded_ohe =pd.DataFrame(view_encoded, columns=ohe.get_feature_names(['view']), index=housing_pred.index)
housing_pred5 = pd.concat([housing_pred4, view_encoded_ohe], axis = 1)

In [16]:
floors_ohe = housing_pred[['floors']]
ohe_floors = OneHotEncoder(categories ='auto', sparse =False)
ohe_floors.fit(floors_ohe)
ohe_floors_encoded = ohe_floors.transform(floors_ohe)
floors_encoded_ohe = pd.DataFrame(ohe_floors_encoded, columns = ohe_floors.get_feature_names(['floors']), index = housing_pred.index )
housing_pred6 = pd.concat([housing_pred5, floors_encoded_ohe ], axis =1)

In [17]:
zipcode_ohe = housing_pred[['zipcode']]
ohe = OneHotEncoder(categories="auto", sparse=False)
ohe.fit(zipcode_ohe)
zipcode_encoded = ohe.transform(zipcode_ohe)
zipcode_encoded_ohe =pd.DataFrame(zipcode_encoded, columns=ohe.get_feature_names(['zipcode']), index=housing_pred.index)
housing_pred_final = pd.concat([housing_pred6, zipcode_encoded_ohe ], axis =1)

In [18]:
housing_pred_final.columns

Index(['id', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition',
       ...
       'zipcode_98146', 'zipcode_98148', 'zipcode_98155', 'zipcode_98166',
       'zipcode_98168', 'zipcode_98177', 'zipcode_98178', 'zipcode_98188',
       'zipcode_98198', 'zipcode_98199'],
      dtype='object', length=152)

In [19]:
housing_pred_final.drop(['floors', 'bedrooms', 'bathrooms', 'view', 'condition', 'zipcode', 'grade_num'], axis = 1, inplace=True)

In [20]:
housing_pred_final.columns

Index(['id', 'price', 'sqft_living', 'sqft_lot', 'waterfront', 'age_when_sold',
       'Basement', 'bedrooms_1', 'bedrooms_2', 'bedrooms_3',
       ...
       'zipcode_98146', 'zipcode_98148', 'zipcode_98155', 'zipcode_98166',
       'zipcode_98168', 'zipcode_98177', 'zipcode_98178', 'zipcode_98188',
       'zipcode_98198', 'zipcode_98199'],
      dtype='object', length=145)

In [21]:
housing_pred_final['log_price'] = np.log(housing_pred_final['price'])

In [22]:
housing_pred_final.corr()

,id,price,sqft_living,sqft_lot,waterfront,age_when_sold,Basement,bedrooms_1,bedrooms_2,bedrooms_3,...,zipcode_98148,zipcode_98155,zipcode_98166,zipcode_98168,zipcode_98177,zipcode_98178,zipcode_98188,zipcode_98198,zipcode_98199,log_price
id,1.000000,-0.016772,-0.012241,-0.131911,-0.003599,-0.021458,0.002585,-0.014019,0.007455,-0.014160,...,0.017864,0.008483,-0.020345,-0.053707,-0.024527,-0.011133,-0.002729,0.022782,-0.049660,-0.003751
price,-0.016772,1.000000,0.701917,0.089876,0.264306,-0.053890,0.180082,-0.057847,-0.144740,-0.184059,...,-0.035762,-0.046074,-0.022561,-0.091703,0.040503,-0.069286,-0.054438,-0.074064,0.083688,0.891745
sqft_living,-0.012241,0.701917,1.000000,0.173453,0.104637,-0.318592,0.204324,-0.124270,-0.350461,-0.273109,...,-0.026323,-0.046455,-0.001475,-0.074826,0.028933,-0.042364,-0.024066,-0.041815,0.010832,0.695185
sqft_lot,-0.131911,0.089876,0.173453,1.000000,0.021459,-0.052853,-0.035227,0.002243,-0.027755,-0.015102,...,-0.007625,-0.016883,-0.003925,-0.010386,-0.008433,-0.018171,-0.009560,-0.012657,-0.028480,0.100025
waterfront,-0.003599,0.264306,0.104637,0.021459,1.000000,0.024406,0.040061,0.015939,0.015811,-0.014085,...,-0.004244,0.007887,0.064380,-0.009265,-0.003787,0.037313,-0.006567,0.035501,-0.005371,0.170720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zipcode_98178,-0.011133,-0.069286,-0.042364,-0.018171,0.037313,0.059367,0.022552,-0.001685,0.022193,-0.012892,...,-0.005701,-0.016092,-0.012089,-0.012445,-0.012113,1.000000,-0.008822,-0.012700,-0.013525,-0.102879
zipcode_98188,-0.002729,-0.054438,-0.024066,-0.009560,-0.006567,0.015064,-0.001731,-0.007618,0.001087,-0.002190,...,-0.004095,-0.011560,-0.008684,-0.008940,-0.008702,-0.008822,1.000000,-0.009123,-0.009716,-0.080068
zipcode_98198,0.022782,-0.074064,-0.041815,-0.012657,0.035501,0.016741,-0.007569,0.014929,0.012526,0.007919,...,-0.005896,-0.016642,-0.012503,-0.012871,-0.012528,-0.012700,-0.009123,1.000000,-0.013988,-0.112345
zipcode_98199,-0.049660,0.083688,0.010832,-0.028480,-0.005371,0.060139,0.102030,0.000500,0.028241,-0.007111,...,-0.006279,-0.017723,-0.013315,-0.013707,-0.013341,-0.013525,-0.009716,-0.013988,1.000000,0.101801


In [32]:
c = housing_pred_final.corr().abs()

s = c.unstack()
so = s.sort_values(kind="quicksort")

print (so[-155:-153])

log_price    sqft_living    0.695185
sqft_living  price          0.701917
dtype: float64
